In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import pygame
from sklearn.metrics import silhouette_samples, silhouette_score

In [67]:
data = pd.read_csv('C://Shaurya Games//New folder//fashion-mnist_train.csv')

data = np.array(data)
m, n = data.shape

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [68]:
def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_deriv(Z):
    return Z > 0

In [69]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

def der_sigmoid(x):
  return sigmoid(x) * (1- sigmoid(x))

In [70]:
def softmax(A):
    
    expA = np.exp(A)
    return expA / sum(np.exp(A))

In [71]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [72]:
def init_params():
    W1 = np.random.rand(10, 784) * 1/math.sqrt(784)
    b1 = np.random.rand(10, 1) * 1/math.sqrt(784)
    W2 = np.random.rand(10, 10) * 1/math.sqrt(10)
    b2 = np.random.rand(10, 1) * 1/math.sqrt(10)
    W3 = np.random.rand(10, 10) * 1/math.sqrt(10)
    b3 = np.random.rand(10, 1) * 1/math.sqrt(10)
        
    return W1, b1, W2, b2, W3, b3
    
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3
    
def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3
    
def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
        
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3  
    return W1, b1, W2, b2, W3, b3
    
def get_predictions(A):
    return np.argmax(A, 0)

In [73]:
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size


In [74]:
def compute_loss(y_true, y_pred):
        loss = -np.sum(y_true * np.log(y_pred + 1e-10)) / len(y_true)
        return loss

In [75]:
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
            print(compute_loss(Y, predictions))
    return W1, b1, W2, b2, W3, b3

In [76]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.1, 650)

Iteration:  0
0.1001864406779661
-4.5011698637287994e-10
Iteration:  10
0.10247457627118645
-3.324631700953274
Iteration:  20
0.10557627118644068
-0.8641583527690774
Iteration:  30
0.10950847457627119
0.28037231241028615
Iteration:  40
0.11216949152542373
0.6858797754231398
Iteration:  50
0.11435593220338983
0.18986884037052434
Iteration:  60
0.18384745762711865
-1.5691903236364402
Iteration:  70
0.27859322033898304
-4.197847017696903
Iteration:  80
0.3365593220338983
-2.6174780774438577
Iteration:  90
0.44364406779661014
7.170291568794447
Iteration:  100
0.4636779661016949
7.656682879337947
Iteration:  110
0.4723389830508475
7.47229116340986
Iteration:  120
0.4780508474576271
7.10192910458279
Iteration:  130
0.48403389830508475
6.62129145243776
Iteration:  140
0.49106779661016947
5.897956976798255
Iteration:  150
0.5140338983050847
4.2421953440163715
Iteration:  160
0.5450508474576271
1.6244434574111697
Iteration:  170
0.498728813559322
3.87863713608173
Iteration:  180
0.5232203389830

In [77]:
z1,a1,z2,a2,z3,a3=forward_prop(W1, b1, W2, b2, W3, b3, X_dev)
predictions=get_predictions(a3)
get_accuracy(predictions, Y_dev)

0.772

In [78]:
df3 = pd.read_csv('C://Shaurya Games//New folder//fashion-mnist_test.csv')

df3 = np.array(df3)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

df3 = df3.T
Y_test = df3[0]
X_test = df3[1:n]
print(X_test)
X_test = X_test / 255.
_,m_train = X_test.shape

z1,a1,z2,a2,z3,a3=forward_prop(W1, b1, W2, b2, W3, b3, X_test)
predictions=get_predictions(a3)
get_accuracy(predictions, Y_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 3 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


0.7804